<a href="https://colab.research.google.com/github/brunobobadilla06/Entrega-de-Proyecto/blob/main/Primera_entrega_CIENCIA_DE_DATOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Presentación del proyecto**
Este proyecto explora un e-commerce de tecnología con foco en el desempeño comercial y operativo de sus productos. Trabajamos con un dataset sintético que simula 25 registros de artículos (notebooks, smartphones, periféricos, almacenamiento, etc.) con información clave: precio, stock, región, fecha de ingreso, satisfacción de clientes y costo operativo.

La motivación es practicar un flujo de trabajo reproducible de análisis exploratorio en Python, siguiendo la consigna de “Visualizaciones en Python”.


Los objetivos del EDA son:

1. conocer la distribución de precios, stock y satisfacción;

2. detectar valores faltantes y atípicos;

3. relacionar precio ↔ satisfacción ↔ stock (gráfico multivariado);

4. obtener evidencia inicial para hipótesis de negocio (p.ej., “¿precios más altos reducen la satisfacción?” o “¿stocks altos se asocian a menor satisfacción por exceso de oferta?”).


**Contexto Comercial y Analítico**
Contexto: tienda online de tecnología en Argentina; se evalúan productos, regiones de venta y costos.

Métricas clave: precio, stock, satisfacción del cliente (1-5), costo operativo.

Decisiones posibles: ajustar precios, reponer stock por región, revisar productos con costo operativo alto vs. baja satisfacción.

**Preguntas/Hipótesis a Resolver mediante el Análisis de Datos**
P1. ¿Cómo se distribuye el precio y qué tan disperso es?

P2. ¿Existe relación entre precio y satisfacción del cliente? (H0: no hay relación; H1: a mayor precio, menor satisfacción).

P3. ¿El stock influye en la satisfacción percibida? (H1: inventarios muy altos podrían no mejorar la satisfacción).

P4. ¿Hay outliers en precio/stock/costo operativo que requieran tratamiento?

P5. ¿Qué regiones muestran mayor presencia en el catálogo y cómo varían las métricas?

**Objetivo**
ealizar un EDA que conecte visualizaciones y resúmenes numéricos con las preguntas/hypótesis, dejando el entorno listo para la siguiente etapa del proyecto final.

# **Lectura de datos**

**Librerias necesarias para el analisis de datos**


**Configuración del entorno de Python en Google Colab**

**Lectura del dataset**

In [ ]:
# Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Mostrar todas las columnas en outputs
pd.set_option('display.max_columns', None)
datos_csv = "https://raw.githubusercontent.com/brunobobadilla06/Entrega-de-Proyecto/refs/heads/main/dataset_ecommerce_tecnologia.csv"

df = pd.read_csv(datos_csv)
df.head()

,id,nombre_producto,fecha_ingreso,satisfaccion_cliente,precio,stock,region,costo_operativo
0,1,"Notebook 14""",2025/04/01,3.4,61065.78,389,Mendoza,25413.37
1,2,"Notebook 15""",2025/10/13,3.2,876904.86,53,Patagonia,255762.00
2,3,Smartphone 128GB,2025/07/30,3.9,224838.78,105,CABA,41792.64
3,4,Smartphone 256GB,2025/05/31,NaN,99733.66,259,Mendoza,36300.40
4,5,Auriculares BT,2025/04/16,4.5,564179.69,309,Patagonia,213392.80


**Análisis inicial del dataset**

In [ ]:
print("Forma (filas, columnas):", df.shape)
print("\nTipos de datos:\n", df.dtypes)
print("\nValores nulos por columna:\n", df.isna().sum())
df.describe(include='all')

**Descripción de variables**
1. id (int): identificador único.

2. nombre_producto (objeto): nombre del artículo.

3. fecha_ingreso (objeto → fecha): fecha de ingreso al catálogo (formato aaaa/mm/dd).

4. satisfaccion_cliente (float): escala 1–5, puede tener nulos.

5. precio (float): precio unitario (ARS en este ejemplo).

6. stock (int): unidades disponibles.

7. region (objeto): región comercial de referencia.

8. costo_operativo (float): costos asociados; puede tener nulos.

# **Data Wrangling - Limpieza y transformación de datos**

**Valores duplicados**

In [ ]:
duplicados = df.duplicated().sum()
print("Filas duplicadas:", duplicados)

# Si hubiera, se eliminan:
# df = df.drop_duplicates().reset_index(drop=True)


**Valores nulos**

In [ ]:
# Vemos nulos
df.isna().sum()

# Ejemplo: imputar satisfacción y costo_operativo con la mediana
df['satisfaccion_cliente'] = df['satisfaccion_cliente'].fillna(df['satisfaccion_cliente'].median())
df['costo_operativo'] = df['costo_operativo'].fillna(df['costo_operativo'].median())

df.isna().sum()


**Tratamiento de valores outliers**

In [ ]:
Q1 = df['precio'].quantile(0.25)
Q3 = df['precio'].quantile(0.75)
IQR = Q3 - Q1

lim_inf = Q1 - 1.5*IQR
lim_sup = Q3 + 1.5*IQR

outliers_precio = df[(df['precio'] < lim_inf) | (df['precio'] > lim_sup)]
print("Outliers en precio:", outliers_precio.shape[0])

# Estrategia simple (solo si lo necesitás): winsorizar a límites IQR
# df['precio'] = np.where(df['precio'] < lim_inf, lim_inf,
#                 np.where(df['precio'] > lim_sup, lim_sup, df['precio']))


**Fechas y columnas derivadas**

In [ ]:
df['fecha_ingreso'] = pd.to_datetime(df['fecha_ingreso'], format="%Y/%m/%d", errors='coerce')
df['anio'] = df['fecha_ingreso'].dt.year
df['mes']  = df['fecha_ingreso'].dt.month
df[['fecha_ingreso','anio','mes']].head()


# **Análisis Exploratorio de datos**

**Análisis univariado**

In [ ]:
plt.figure()
plt.hist(df["precio"].dropna(), bins=10)
plt.title("Distribución de precio")
plt.xlabel("Precio")
plt.ylabel("Frecuencia")
plt.show()


**Análisis bivariado**

In [ ]:
plt.figure()
mask = df["satisfaccion_cliente"].notna()
plt.scatter(df.loc[mask, "precio"], df.loc[mask, "satisfaccion_cliente"])
plt.title("Precio vs Satisfacción del cliente")
plt.xlabel("Precio")
plt.ylabel("Satisfacción del cliente")
plt.show()


**Análisis multivariado**

In [ ]:
plt.figure()
sizes = (df.loc[mask, "stock"] + 10)   # evita tamaño cero
plt.scatter(df.loc[mask, "precio"], df.loc[mask, "satisfaccion_cliente"], s=sizes)
plt.title("Precio vs Satisfacción (tamaño ~ Stock)")
plt.xlabel("Precio")
plt.ylabel("Satisfacción del cliente")
plt.show()


**Conclusiones del EDA**
P1 (distribución de precio): la distribución muestra rango amplio; conviene reportar mediana y cuartiles.

P2 (precio vs satisfacción): con 25 muestras no se observa patrón fuerte; no hay evidencia clara de relación lineal.

P3 (stock): al incorporarlo como tamaño, no se ve una tendencia obvia; podría explorarse por región en la próxima entrega.

Nulos: se detectaron 2 y se imputaron con mediana (criterio simple para la práctica).

Outliers: detectados con IQR; decidir tratamiento (winsorizar o investigar puntualmente).

Próximos pasos: agregar ventas/margen y series de tiempo por mes/región para validar hipótesis con mayor soporte.